In [2]:
import twitter

def oauth_login():
    # XXX: Go to  to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
    # for more information on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'QyEuEeelcKfWtkDjIYz0uDlYB'
    CONSUMER_SECRET = 'QNaXh3s8dBZs0otrbFw1h6yz5jIYnAjEUFqVsq65hVQYwjaYU2'
    OAUTH_TOKEN = '389829220-3rB90XkBtzuWXjSiG9gnTQ4hYfEYtOIogRgHynaH'
    OAUTH_TOKEN_SECRET = 'KnTDcaZ6DkKbUXE10IOZauyGVYnbVcj29OiAYsmRCGEeD'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

{
 "created_at": "Mon May 20 09:08:41 +0000 2019",
 "id": 1130399937416650758,
 "id_str": "1130399937416650758",
 "text": "RT @Mond_Rakete: Take a look at $TERA. A truly innovative gem with x100 potential or more.\nThink $NANO with smart contracts and dapps. Esse\u2026",
 "truncated": false,
 "entities": {
  "hashtags": [],
  "symbols": [
   {
    "text": "TERA",
    "indices": [
     32,
     37
    ]
   },
   {
    "text": "NANO",
    "indices": [
     97,
     102
    ]
   }
  ],
  "user_mentions": [
   {
    "screen_name": "Mond_Rakete",
    "name": "Mondrakete",
    "id": 2506088719,
    "id_str": "2506088719",
    "indices": [
     3,
     15
    ]
   }
  ],
  "urls": []
 },
 "metadata": {
  "iso_language_code": "en",
  "result_type": "recent"
 },
 "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
 "in_reply_to_status_id": null,
 "in_reply_to_status_id_str": null,
 "in_reply_to_user_id": null,
 "in_reply_to_user_id_str": null,
 

In [58]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [57]:
from textblob import TextBlob
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [46]:
def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
    # and https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators
    # for details on advanced search criteria that may be useful for 
    # keyword arguments
    
    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://developer.twitter.com/en/docs/basics/rate-limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    # Read this https://developer.twitter.com/en/docs/tweets/timelines/guides/working-with-timelines to understand why we use 
    # max_id to populate tweets
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

In [62]:
twitter_api = oauth_login()
stock_names = ['$TERA', '$ARMC', '$GTMM', '$GNCP', '$GLAE', '$CRAY', '$GPRO', '$SWCH', '$SCKT']
for stock in stock_names:
    q = stock
    results = twitter_search(twitter_api, q, max_results=100)
    positive = 0
    negative = 0
    for i in results:
        sentiment = get_tweet_sentiment(i['text'])
        if sentiment == 'positive':
            positive += 1
        elif sentiment == 'negative':
            negative += 1
    print('Positive tweets for {} is {} %'.format(stock, (positive/len(results))*100))
    print('Negative tweets for {} is {} %'.format(stock, (negative/len(results))*100))

Positive tweets for $TERA is 66.66666666666666 %
Negative tweets for $TERA is 0.0 %
Positive tweets for $ARMC is 45.3125 %
Negative tweets for $ARMC is 7.03125 %
Positive tweets for $GTMM is 0.0 %
Negative tweets for $GTMM is 0.0 %
Positive tweets for $GNCP is 50.0 %
Negative tweets for $GNCP is 8.5 %
Positive tweets for $GLAE is 0.0 %
Negative tweets for $GLAE is 0.0 %
Positive tweets for $CRAY is 46.15384615384615 %
Negative tweets for $CRAY is 5.641025641025641 %
Positive tweets for $GPRO is 40.32258064516129 %
Negative tweets for $GPRO is 9.67741935483871 %
Positive tweets for $SWCH is 25.675675675675674 %
Negative tweets for $SWCH is 9.45945945945946 %
Positive tweets for $SCKT is 33.33333333333333 %
Negative tweets for $SCKT is 8.333333333333332 %
